In [1]:
import sys
sys.path.append('../../hw/base') # just for the boilerplate stuff
sys.path.append('../../hw/cpu') 

from verification_utils import CHCs, mk_int_array
import boilerplate
from instruction_set import *
from assembler import asm, disasm, disasm_pretty, asm_bin


In [2]:
from z3 import *
import z3
z3.set_param(proof=True)  # this is needed so that we can call `.proof()` later

In [3]:
stack = Array('stack', IntSort(), IntSort())
sp = Int('sp')
r0 = Int('r0')
r1 = Int('r1')

state_vars = [stack, sp, r0, r1]


a, b = input_vars = Ints('a b')

In [4]:
instrList = [('DUP', 1), ('DUP', 1), 'mod', ('DUP', 1), ('DUP', 1), ('POP', 2), ('ALU', 'LT'), ('POP', 1), ('JZ', '_else_0'), ('DUP', 1), ('JMP', '_end_0'),
             '_else_0', ('DUP', 1), ('DUP', 1), ('POP', 2), ('ALU', 'SUB'), ('DUP', 1), ('YANK', (2, 2)), ('JMP', 'mod'), '_end_0', ('YANK', (1, 2))]

# function a mod b from snake
'''
a mod b

mod(2) = if $0 < $1 then $0 else .mod ($0 - $1) $1
'''

asm_list = asm(instrList, start_addr=0)
dict= asm_list.label_addrs

sigma = [*state_vars]
U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort()) for i in range(len(instrList) + 1)}

# init state of stack and sp, stack[a, b]
q1 = And(stack[sp - 1] > 0, stack[sp - 2] > 0)
# final state of the vars
q2 = stack[sp - 1] == stack[sp - 3]%stack[sp - 2]

In [5]:
instrList = ['checkHeadUp', ('DUP', 0), ('PUSH', 40960), ('POP', 2), ('ALU', 'LT'), ('POP', 1), ('JZ', '_else_0'), ('PUSH', 1), ('JMP', '_end_0'),
             '_else_0', ('PUSH', 40975), ('DUP', 1), ('POP', 2), ('ALU', 'LT'), ('POP', 1), ('JZ', '_else_1'), ('PUSH', 1),
             ('JMP', '_end_1'), '_else_1', ('PUSH', 0), '_end_1', '_end_0']

# checkHeadUp from snake
'''
0xa000 <= $0 <= 0xa00f --> return 0 else return 1

checkHeadUp(1) = if $0 < 0xa000 then 1 else (if 0xa00f < $0 then 1 else 0)
'''

asm_list = asm(instrList, start_addr=0)
dict= asm_list.label_addrs

sigma = [*state_vars]
U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort()) for i in range(len(instrList) + 1)}

# init state of stack and sp, stack[a]
q1 = And(stack[sp - 1] >= 0xa000, stack[sp - 1] <= 0xa00f) 
# final state of the vars
q2 = stack[sp - 1] == 0

In [6]:
instrList = ['find_max', ('DUP', 1), ('POP', 1), ('LOAD', 0), 'find_max_aux', ('PUSH', 1), ('DUP', 2),
             ('POP', 2), ('ALU', 'LT'), ('POP', 1), ('JZ', '_else_0'), ('DUP', 0), ('DUP', 3), ('DUP', 3), ('POP', 2), ('ALU', 'ADD'),
             ('PUSH', 1), ('POP', 2), ('ALU', 'SUB'), ('POP', 1), ('LOAD', 0), ('POP', 2), ('ALU', 'LT'), ('POP', 1), ('JZ', '_else_1'), ('DUP', 2), 
             ('DUP', 2), ('PUSH', 1), ('POP', 2), ('ALU', 'SUB'), ('DUP', 4), ('DUP', 4), ('POP', 2), ('ALU', 'ADD'), ('PUSH', 1), ('POP', 2),
             ('ALU', 'SUB'), ('POP', 1), ('LOAD', 0), ('YANK', (3, 3)), ('JMP', 'find_max_aux'),  '_else_1', ('DUP', 2), ('DUP', 2),
             ('PUSH', 1), ('POP', 2), ('ALU', 'SUB'), ('DUP', 2), ('YANK', (3, 3)), ('JMP', 'find_max_aux'), '_else_0', ('DUP', 0), ('YANK', (1, 2))]

# find max in array
'''
    find_max(2) = mem_peek $0; find_max_aux $0 $1 $-3
    find_max_aux(3) = if 1 < $1 then (if $2 < mem_peek ($0+$1-1) then .find_max_aux $0 ($1-1) (mem_peek ($0+$1-1)) else .find_max_aux $0 ($1 - 1) $2
                                ) else $2
'''


asm_list = asm(instrList, start_addr=0)
dict= asm_list.label_addrs

sigma = [*state_vars]
U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort()) for i in range(len(instrList) + 1)}

# init state of stack and sp, stack[init_adrr, array_length], init_adrr > 0, array_length > 0
'''
stack = [j, init_address_array, array_length] and array_length > 0 and 0 <= j < array_length and init_address_array >= 0
'''
q1 = And(stack[sp - 2] >= sp + 15, stack[sp - 3] >= 0, stack[sp - 3] < stack[sp - 1]) 
# final state of the vars
'''
stack = [j, init_address_array, max_value] ^ stack[init_address_array + j] <= max_value
'''
q2 = stack[sp - 1] >= stack[stack[sp - 2] + stack[sp - 3]]

In [7]:
instrList = ['div_mod', ('DUP', 0), ('PUSH', 0), 'div_mod_aux', ('DUP', 1), ('DUP', 4), ('POP', 2), ('ALU', 'LT'), ('POP', 1), ('JZ', '_else_0'),
             ('DUP', 0), ('JMP', 'exit'), '_else_0', ('DUP', 1), ('DUP', 4), ('POP', 2), ('ALU', 'SUB'),
             ('DUP', 1), ('PUSH', 1), ('POP', 2), ('ALU', 'ADD'), ('YANK', (2, 2)), ('JMP', 'div_mod_aux'), 'exit', ('POP', 1)]

# div_mod (THIS IS DIV and MOD) general
'''
div_mod(2) = div_mod_aux $0 $1 $1 0
div_mod_aux(4) = if $2 < $0 then ($3) 
                        else div_mod_aux $0 $1 ($2 - $0) ($3 + 1)
'''

asm_list = asm(instrList, start_addr=0)
dict= asm_list.label_addrs

sigma = [*state_vars]
U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort()) for i in range(len(instrList) + 1)}

# init state of stack and sp, stack = [a, b] and a > 0
q1 = stack[sp - 2] > 0
# final state of the vars, stack = [a, b, mod, div]
# unsat because sat is taking too long to load!
q2 = stack[sp - 3] == (stack[sp - 4]*stack[sp - 1] + stack[sp - 2])

In [8]:
instrList = [ 'div_mod_3', ('DUP', 0), ('PUSH', 0), 'div_mod_aux', ('DUP', 1), ('PUSH', 3), ('POP', 2), ('ALU', 'LT'),
 ('POP', 1), ('JZ', '_else_0'), ('JMP', '_end_0'), '_else_0', ('DUP', 1), ('DUP', 2), ('PUSH', 3), 
 ('POP', 2), ('ALU', 'SUB'), ('DUP', 1), ('PUSH', 1), ('POP', 2), ('ALU', 'ADD'), ('YANK', (2, 2)), ('JMP', 'div_mod_aux'), '_end_0']

# div_mod (THIS IS DIV and MOD) general with a const denominator 
'''
div_mod_3(1) = div_mod_aux $0 $0 0
div_mod_aux(3) = if $1 < 3 then $2
                           else .div_mod_aux $0 ($1 - 3) ($2 + 1)


div_mod_3: 
    DUP 0
    PUSH 0    
div_mod_aux (loop):
    DUP 1; PUSH 3; POP 2; ALU LT; POP 1; JZ _else_0
    JMP _end_0
_else_0:
    DUP 1
    DUP 2; PUSH 3; POP 2; ALU SUB
    DUP 1; PUSH 1; POP 2; ALU ADD
    YANK 2 2
    JMP div_mod_aux (loop)
_end_0:

'''

asm_list = asm(instrList, start_addr=0)
dict= asm_list.label_addrs

sigma = [*state_vars]
U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort()) for i in range(len(instrList) + 1)}

# init state of stack and sp, stack = [b] and b > 0
q1 = stack[sp - 1] == 3
# final state of the vars, stack = [b, mod, div], b = 3*div + mod
# unsat because sat is taking too long to load!
q2 = stack[sp - 3] == (3*stack[sp - 1] + stack[sp - 2])

In [9]:
"""
{  stack= [a, b] ∧ b > 0 }
  DUP 1
  PUSH 0
loop:
  DUP; DUP 3; POP 2; ALU LT; POP 1; JZ exit
  DUP 1; PUSH 1; POP 2; ALU ADD
  DUP 1; PUSH 1; POP 2; ALU ADD
  YANK 2,2
  JMP loop
exit:
  POP 1
{ stack = [a, b, a + b] }
"""

# a + b

instrList= [('DUP', 1), ('PUSH', 0), 'loop', ('DUP', 0), ('DUP', 3), ('POP', 2), ('ALU', 'LT'), ('POP', 1), ('JZ', 'exit'),
            ('DUP', 1), ('PUSH', 1), ('POP', 2), ('ALU', 'ADD'), ('DUP', 1), ('PUSH', 1), ('POP', 2), ('ALU', 'ADD'), ('YANK', (2, 2)), ('JMP', 'loop'), 'exit', ('POP', 1)]

asm_list = asm(instrList, start_addr=0)
dict= asm_list.label_addrs

sigma = [*state_vars]
U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort()) for i in range(len(instrList) + 1)}

# init state of stack and sp, stack[a, b]
q1 = stack[sp - 1] > 0
# final state of the vars
q2 = False



In [10]:
    sigma = [*state_vars]
    U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort()) for i in range(len(instrList) + 1)}
    chcs_to_solve = [Implies(q1, U[0](sigma))]
    addresses_list = []
    for i in range(len(instrList)):
        #print(instrList[i])
        #print(type(instrList[i]))
        if isinstance(instrList[i], str):
            to_add = Implies(U[i](sigma), U[i+1](sigma))
            chcs_to_solve.append(to_add)
        else:
            instr, inp = instrList[i]
            if instr == 'PUSH':
                if isinstance(inp, str):
                    to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, instrList.index(inp)), sp + 1, r0, r1))
                    addresses_list.append(instrList.index(inp))
                else: to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, inp), sp + 1, r0, r1))
                chcs_to_solve.append(to_add)
                continue
            elif instr == 'POP':
                if inp == 1:
                    to_add = Implies(U[i](sigma), U[i+1](stack, sp - 1, stack[sp - 1], r1))
                    chcs_to_solve.append(to_add)
                    continue  
                elif inp == 2:
                    to_add = Implies(U[i](sigma), U[i+1](stack, sp - 2, stack[sp - 2], stack[sp - 1]))
                    chcs_to_solve.append(to_add)
                    continue  
            elif instr == 'DUP':
                to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, stack[sp - inp - 1]), sp + 1, r0, r1))
                chcs_to_solve.append(to_add)
                continue                  
            elif instr == 'YANK':
                tmp = stack
                x, y = inp
                k = 0
                j = y
                while k < y:
                    tmp = Store(tmp, sp - x - j, stack[sp - x + k])
                    k = k + 1
                    j = j - 1
                to_add = Implies(U[i](sigma), U[i+1](tmp, sp - y, r0, r1))
                chcs_to_solve.append(to_add)
                continue                                      
            elif instr == 'STOR':
                to_add = Implies(U[i](sigma), U[i+1](Store(stack, r1, r0), sp, r0, r1))
                chcs_to_solve.append(to_add)
                continue  
            elif instr == 'LOAD':
                to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, stack[r0]), sp + 1, r0, r1))
                chcs_to_solve.append(to_add)
                continue  
            elif instr == 'JMP':
                to_add = Implies(U[i](sigma), U[instrList.index(inp) + 1](sigma))
                chcs_to_solve.append(to_add)
                continue                  
            elif instr == 'JZ':
                to_add = Implies(And(U[i](sigma), r0 != 0), U[i+1](sigma))
                chcs_to_solve.append(to_add)
                to_add = Implies(And(U[i](sigma), r0 == 0), U[instrList.index(inp) + 1](sigma))
                chcs_to_solve.append(to_add)
                continue                  
            elif instr == 'JNZ':
                to_add = Implies(And(U[i](sigma), r0 == 0), U[i+1](sigma))
                chcs_to_solve.append(to_add)
                to_add = Implies(And(U[i](sigma), r0 != 0), U[instrList.index(inp) + 1](sigma))
                chcs_to_solve.append(to_add)
                continue
            elif instr == 'RET':  
                print(addresses_list)
                if inp == 0:
                    addrs = addresses_list.pop()
                    to_add = Implies(And(U[i](sigma), r0 != instrList.index('finish')), U[addrs+1](sigma))
                    chcs_to_solve.append(to_add)
                    addrs = addresses_list.pop()
                    to_add = Implies(And(U[i](sigma), r0 == instrList.index('finish')), U[addrs+1](sigma))
                    chcs_to_solve.append(to_add)
                    continue
                elif inp == 1:
                    addrs = addresses_list.pop()
                    to_add = Implies(And(U[i](sigma), r0 != instrList.index('finish')), U[addrs+1](Store(stack, sp, r1), sp + 1, r0, r1))
                    chcs_to_solve.append(to_add)
                    addrs = addresses_list.pop()
                    to_add = Implies(And(U[i](sigma), r0 == instrList.index('finish')),  U[addrs+1](Store(stack, sp, r1), sp + 1, r0, r1))
                    chcs_to_solve.append(to_add)
                    continue
            elif instr == 'ALU':
                if inp == 'ADD':
                    to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, r0 + r1), sp + 1, r0, r1))
                    chcs_to_solve.append(to_add)
                    continue
                elif inp == 'SUB':
                    to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, r0 - r1), sp + 1, r0, r1))
                    chcs_to_solve.append(to_add)
                    continue
                elif inp == 'MUL':
                    to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, r0*r1), sp + 1, r0, r1))
                    chcs_to_solve.append(to_add)
                    continue
                elif inp == 'SHL':
                    to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, r0 << r1), sp + 1, r0, r1))
                    chcs_to_solve.append(to_add)
                    continue
                elif inp == 'SHR':
                    to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, r0 >> r1), sp + 1, r0, r1))
                    chcs_to_solve.append(to_add)
                    continue
                elif inp == 'NEG':
                    to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, -r0), sp + 1, r0, r1))
                    chcs_to_solve.append(to_add)
                    continue
                elif inp == 'AND':
                    to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, r0 & r1), sp + 1, r0, r1))
                    chcs_to_solve.append(to_add)
                    continue
                elif inp == 'OR':
                    to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, r0 | r1), sp + 1, r0, r1))
                    chcs_to_solve.append(to_add)
                    continue
                elif inp == 'NOT':
                    to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp,  ~r0), sp + 1, r0, r1))
                    chcs_to_solve.append(to_add)
                    continue
                elif inp == 'LT':
                    to_add = Implies(U[i](sigma), U[i+1](Store(stack, sp, r0 < r1), sp + 1, r0, r1))
                    chcs_to_solve.append(to_add)
                    continue
    chcs_to_solve.append(Implies(U[len(instrList)](sigma), q2)) 
    chcs = CHCs(chcs_to_solve)
    chcs

"stack[sp - 1] > 0 ⇒ U0(stack, sp, r0, r1)"
"U0(stack, sp, r0, r1) ⇒ U1(Store(stack, sp, stack[sp - 1 - 1]), sp + 1, r0, r1)"
"U1(stack, sp, r0, r1) ⇒ U2(Store(stack, sp, 0), sp + 1, r0, r1)"
"U2(stack, sp, r0, r1) ⇒ U3(stack, sp, r0, r1)"
"U3(stack, sp, r0, r1) ⇒ U4(Store(stack, sp, stack[sp - 0 - 1]), sp + 1, r0, r1)"
"U4(stack, sp, r0, r1) ⇒ U5(Store(stack, sp, stack[sp - 3 - 1]), sp + 1, r0, r1)"
"U5(stack, sp, r0, r1) ⇒ U6(stack, sp - 2, stack[sp - 2], stack[sp - 1])"
"U6(stack, sp, r0, r1) ⇒ U7(Store(stack, sp, If(r0 < r1, 1, 0)), sp + 1, r0, r1)"
"U7(stack, sp, r0, r1) ⇒ U8(stack, sp - 1, stack[sp - 1], r1)"
"U8(stack, sp, r0, r1) ∧ r0 ≠ 0 ⇒ U9(stack, sp, r0, r1)"
"U8(stack, sp, r0, r1) ∧ r0 = 0 ⇒ U20(stack, sp, r0, r1)"


In [ ]:
s = chcs.create_solver()
#s.set('xform.inline_eager', False)
#s.set('xform.inline_linear', False)
s.set(timeout=10000)
%time s.check()

In [ ]:
#s.model()